In [1]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

/home/ivan/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Давайте проанализируем данные опроса 4361 женщин из Ботсваны:
botswana.tsv

О каждой из них мы знаем:

   * сколько детей она родила (признак ceb)
   * возраст (age)
   * длительность получения образования (educ)
   * религиозная принадлежность (religion)
   * идеальное, по её мнению, количество детей в семье (idlnchld)
   * была ли она когда-нибудь замужем (evermarr)
   * возраст первого замужества (agefm)
   * длительность получения образования мужем (heduc)
   * знает ли она о методах контрацепции (knowmeth)
   * использует ли она методы контрацепции (usemeth)
   * живёт ли она в городе (urban)
   * есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)

Давайте научимся оценивать количество детей ceb по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

In [4]:
raw = pd.read_csv("botswana.tsv", sep="\t", index_col=False) 
raw.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [5]:
print raw.religion.value_counts()

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64


Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски? 

In [8]:
len(raw.dropna())

1834

В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке agefm пропуски стоят только там, где evermarr=0, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо".

В подобных случаях, когда признак x1 на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

   * создать новый бинарный признак
   
        * x2=1,x1='не применимо',
        * 0,иначе;
    
   * заменить "не применимо" в x1 на произвольную константу c, 
    которая среди других значений x1 не встречается.

Теперь, когда мы построим регрессию на оба признака и получим модель вида
y=β0+β1x1+β2x2,
на тех объектах, где x1 было измерено, регрессионное уравнение примет вид
y=β0+β1x,
а там, где x1 было "не применимо", получится
y=β0+β1c+β2.
Выбор c влияет только на значение и интерпретацию β2, но не β1.

Давайте используем этот метод для обработки пропусков в agefm и heduc.

   1. Создайте признак nevermarr, равный единице там, где в agefm пропуски.
   2. Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице X будет мультиколлинеарность.
   3. Замените NaN в признаке agefm на cagefm=0.
   4. У объектов, где nevermarr = 1, замените NaN в признаке heduc на cheduc1=−1 (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).

Сколько осталось пропущенных значений в признаке heduc?

In [73]:
data = raw

In [74]:
data['nevermarr'] = data['agefm'].fillna(1).apply(lambda x : 0 if x > 1 else 1)

In [75]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [76]:
data = data.drop('evermarr', axis = 1)

In [77]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [78]:
data.agefm.fillna(0, inplace = True)

In [79]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [84]:
data.loc[data['nevermarr'] == 1,'heduc']= -1

In [85]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [89]:
print len(data.heduc) - len(data.heduc.dropna())

123


Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения (cidlnchld=−1, cheduc2=−2 (значение -1 мы уже использовали), cusemeth=−1).

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [94]:
data['idlnchld_noans'] = data['idlnchld'].fillna(-1).apply(lambda x : 0 if x > -1 else 1)
data.idlnchld.fillna(-1, inplace = True)

data['heduc_noans'] = data['heduc'].fillna(-2).apply(lambda x : 0 if x > -2 else 1)
data.heduc.fillna(-2, inplace = True)

data['usemeth_noans'] = data['usemeth'].fillna(-1).apply(lambda x : 0 if x > -1 else 1)
data.usemeth.fillna(-1, inplace = True)

In [95]:
data

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0
5,1,30,5,spirit,5.0,1.0,0.0,24.0,7.0,1,1.0,0.0,0.0,0.0,0,0,0,0
6,3,42,4,other,3.0,1.0,0.0,15.0,11.0,1,1.0,0.0,1.0,0.0,0,0,0,0
7,1,36,7,other,4.0,1.0,1.0,24.0,9.0,1,1.0,0.0,0.0,0.0,0,0,0,0
8,4,37,16,catholic,4.0,1.0,1.0,26.0,17.0,1,1.0,1.0,1.0,1.0,0,0,0,0
9,1,34,5,protestant,4.0,1.0,1.0,18.0,3.0,1,0.0,1.0,0.0,0.0,0,0,0,0


In [100]:
data.dropna(inplace = True)

In [104]:
print data.shape, ',', data.shape[0]*data.shape[1]

(4348, 18) , 78264


Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации R2? Округлите до трёх знаков после десятичной точки.

In [105]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio +'\
                'tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted = m1.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Sun, 18 Jun 2017   Prob (F-statistic):               0.00
Time:                        14:33:33   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

In [106]:
print 'Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1]

Breusch-Pagan test: p=0.000000


/home/ivan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `het_breushpagan` is deprecated, use `het_breuschpagan` instead!
Use het_breuschpagan, het_breushpagan will be removed in 0.9 
(Note: misspelling missing 'c')
  """Entry point for launching an IPython kernel.


Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [112]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0


In [115]:
data_cut = data.drop(['religion', 'radio', 'tv'], axis = 1)

In [116]:
data_cut.head()

,ceb,age,educ,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1,0,0,0
1,2,43,11,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,0,0,0,0
2,0,49,4,4.0,1.0,0.0,22.0,1.0,1,1.0,0.0,0,0,0,0
3,0,24,12,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1,0,0,0
4,3,32,13,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,0,0,0,0


In [119]:
m2 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric +'\
                'bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data_cut)
fitted_cut = m2.fit(cov_type='HC1')
print fitted_cut.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Sun, 18 Jun 2017   Prob (F-statistic):               0.00
Time:                        14:47:08   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [120]:
print 'Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted_cut.resid, fitted.model.exog)[1]

Breusch-Pagan test: p=0.000000


/home/ivan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `het_breushpagan` is deprecated, use `het_breuschpagan` instead!
Use het_breuschpagan, het_breushpagan will be removed in 0.9 
(Note: misspelling missing 'c')
  """Entry point for launching an IPython kernel.


In [121]:
print "F=%f, p=%f, k1=%f" % fitted.compare_f_test(fitted_cut)

F=0.919236, p=0.467231, k1=5.000000


/home/ivan/anaconda2/lib/python2.7/site-packages/statsmodels/regression/linear_model.py:1746: InvalidTestWarning: F test for comparison is likely invalid with robust covariance, proceeding anyway
  InvalidTestWarning)


Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением cusemeth=−1, удалять usemeth_noans и usemeth можно только вместе.

Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили 5.5×10−8, нужно ввести 8).

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без usemeth и usemeth_noans. 

In [122]:
data_cut2 = data_cut.drop(['usemeth', 'usemeth_noans'], axis = 1)

In [124]:
data_cut2.head()

,ceb,age,educ,idlnchld,knowmeth,agefm,heduc,urban,electric,bicycle,nevermarr,idlnchld_noans,heduc_noans
0,0,18,10,4.0,1.0,0.0,-1.0,1,1.0,1.0,1,0,0
1,2,43,11,2.0,1.0,20.0,14.0,1,1.0,1.0,0,0,0
2,0,49,4,4.0,1.0,22.0,1.0,1,1.0,0.0,0,0,0
3,0,24,12,2.0,1.0,0.0,-1.0,1,1.0,1.0,1,0,0
4,3,32,13,3.0,1.0,24.0,12.0,1,1.0,1.0,0,0,0


In [125]:
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban + electric +'\
                'bicycle + nevermarr + idlnchld_noans + heduc_noans', 
             data=data_cut2)
fitted_cut2 = m3.fit(cov_type='HC1')
print fitted_cut.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Sun, 18 Jun 2017   Prob (F-statistic):               0.00
Time:                        14:55:30   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [133]:
temp = fitted_cut.compare_f_test(fitted_cut2)

/home/ivan/anaconda2/lib/python2.7/site-packages/statsmodels/regression/linear_model.py:1746: InvalidTestWarning: F test for comparison is likely invalid with robust covariance, proceeding anyway
  InvalidTestWarning)


In [137]:
temp[1]

3.1552009480371243e-40

In [132]:
print "F=%f, p=%f, k1=%f" % fitted_cut.compare_f_test(fitted_cut2)

F=92.890582, p=0.000000, k1=2.000000


/home/ivan/anaconda2/lib/python2.7/site-packages/statsmodels/regression/linear_model.py:1746: InvalidTestWarning: F test for comparison is likely invalid with robust covariance, proceeding anyway
  InvalidTestWarning)
